In [4]:
from PyPDF2 import PdfReader, PdfWriter
import numpy as np
import cohere
import re
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
load_dotenv()

True

In [2]:
def extract_pages(input_pdf, output_pdf, pages_to_extract):
    # Create a PDF reader and writer
    reader = PdfReader(input_pdf)
    writer = PdfWriter()
    
    # Extract the specified pages
    for page_num in pages_to_extract:
        writer.add_page(reader.pages[page_num])

    # Write the extracted pages to a new PDF
    with open(output_pdf, 'wb') as output_file:
        writer.write(output_file)

    print(f"New PDF created: {output_pdf}")


# input_pdf = r'D:\Programming\AI\Basics\AMNIL Tech\Chat With Docs\energy.pdf' 
# output_pdf = 'sample.pdf'       
# pages_to_extract = [0,1]           

# extract_pages(input_pdf, output_pdf, pages_to_extract)


In [5]:
pdf_path = 'sir_doc_file.pdf'
reader = PdfReader(pdf_path)
page = reader.pages[0]

In [6]:
reader.pages

In [7]:
parts = []
text_body = []
text = ''
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]        
    if y > 50 and y < 800:
        #print(f"Text: {repr(text)}, Font Size: {fontSize}")
        parts.append(text)
for page in reader.pages:
    page.extract_text(visitor_text=visitor_body)
    text_body.append(''.join(parts))
text = ''.join(text_body)
    

In [8]:
def chunk_by_sentence(text, max_length):
    # Regular expression to match sentence-ending punctuation: . ? !
    sentence_endings = re.compile(r'([.!?])')
    text = re.sub(r'\s+', ' ', text).strip()
    # Split the text into sentences based on the sentence-ending punctuation
    sentences = sentence_endings.split(text)
    
    # Recombine punctuation with the sentences
    sentences = [sentences[i] + (sentences[i+1] if i+1 < len(sentences) else '') for i in range(0, len(sentences), 2)]
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # If adding the sentence exceeds max_length, start a new chunk
        if len(current_chunk) + len(sentence) > max_length:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence.strip()
        else:
            current_chunk += sentence  # Add the sentence to the current chunk

    if current_chunk:  # Append the last chunk
        chunks.append(current_chunk)
    
    return chunks

In [9]:
chunks = chunk_by_sentence(text,500)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
HybridRAG: Integrating Knowledge Graphs and Vector Retrieval Augmented Generation for Efficient Information Extraction Bhaskarjit Sarmah bhaskarjit.sarmah@blackrock.com BlackRock, Inc. Gurugram, IndiaBenika Hall bhall@nvidia.com NVIDIA Santa Clara, CA, USARohan Rao rohrao@nvidia.com NVIDIA Santa Clara, CA, USA Sunil Patel supatel@nvidia.com NVIDIA Santa Clara, CA, USAStefano Pasquali stefano.pasquali@blackrock.com BlackRock, Inc. New York, NY, USADhagash Mehta dhagash.mehta@blackrock.

Chunk 2:
com BlackRock, Inc.

Chunk 3:
New York, NY, USA ABSTRACT Extraction and interpretation of intricate information from unstruc- tured text data arising in financial applications, such as earnings call transcripts, present substantial challenges to large language models (LLMs) even using the current best practices to use Re- trieval Augmented Generation (RAG) (referred to as VectorRAG techniques which utilize vector databases for information retrieval) due to challenges such as domain spec

In [10]:
chunks

['HybridRAG: Integrating Knowledge Graphs and Vector Retrieval Augmented Generation for Efficient Information Extraction Bhaskarjit Sarmah bhaskarjit.sarmah@blackrock.com BlackRock, Inc. Gurugram, IndiaBenika Hall bhall@nvidia.com NVIDIA Santa Clara, CA, USARohan Rao rohrao@nvidia.com NVIDIA Santa Clara, CA, USA Sunil Patel supatel@nvidia.com NVIDIA Santa Clara, CA, USAStefano Pasquali stefano.pasquali@blackrock.com BlackRock, Inc. New York, NY, USADhagash Mehta dhagash.mehta@blackrock.',
 'com BlackRock, Inc.',
 'New York, NY, USA ABSTRACT Extraction and interpretation of intricate information from unstruc- tured text data arising in financial applications, such as earnings call transcripts, present substantial challenges to large language models (LLMs) even using the current best practices to use Re- trieval Augmented Generation (RAG) (referred to as VectorRAG techniques which utilize vector databases for information retrieval) due to challenges such as domain specific terminology an

In [11]:
def calculate_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [12]:
COHERE_KEY = os.getenv('COHERE_KEY')
co = cohere.Client(COHERE_KEY)

In [15]:
len(chunks)

651

In [14]:
chunks = [chunk for chunk in chunks if len(chunk) > 10]

In [16]:
model = "embed-english-v3.0"
input_type = "search_document"

chunk_embeddings = co.embed(
    texts = chunks,
    model = model,
    input_type = input_type,
    embedding_types=['float']
)

In [47]:
questions = ['Who are the writers?','How are knowledge graphs constructed?','What is knowledge extraction?','What are the evaluation metrics?',
             'Which library is used to import pdf documents?','Compare between vector rag and graph rag','Configuration for graphrag']

In [48]:
input_type = 'search_query'
query = questions[-1]
query_embedding = co.embed(
    texts = [query],
    model = model,
    input_type = input_type,
    embedding_types= ['float']
)

In [49]:
chunk_embedding_floats = chunk_embeddings.embeddings.float
query_embedding_float = query_embedding.embeddings.float[0]
similarities = []
for chunk_embedding_float in chunk_embedding_floats:
    similarity = calculate_similarity(chunk_embedding_float,query_embedding_float)
    similarities.append(similarity)


In [50]:
similarities_np = np.array(similarities)

In [51]:
# Input prompt for the model
prompt = f"""
Use only the context provided to answer the following question. If you don't know the answer say you don't know.
Context: {chunks[similarities_np.argmax()]}
Question: {query}
"""

# Generate text
response = co.generate(
    model="command",
    prompt=prompt,
    max_tokens=200,  
    temperature=0.2,  
)

In [52]:
# Access the generated text
print(response.generations[0].text)

 I'm sorry, the provided context did not mention anything about the specific configuration for GraphRAG. Therefore, I don't have any information on this aspect. If there is anything else you would like me to try and answer, please let me know! 


In [74]:
# phrases = ["i love soup", "soup is my favorite", "london is far away"]
# model = "embed-english-v3.0"
# input_type = "search_query"
# res = co.embed(
#     texts=phrases,
#     model=model,
#     input_type=input_type,
#     embedding_types=["float"],
# )
# (soup1, soup2, london) = res.embeddings.float
# calculate_similarity(soup1, soup2)  # 0.85 
# calculate_similarity(soup1, london) # 0.164789

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)